## AutoML Sweepable API

This Notebook shows how to use `Sweepable` API to fully customize the pipeline or search space in your AutoML task. In this notebook, you will learn
- use `AutoTrainer` to simplify your work.
- how to use `AutoML().CreateSweepableEstimator` to create `SweepableEstimator`.
- how to create `SweepablePipeline` for multiple trainer candidates.


### Install Nuget packages and add using statement

In [1]:
//  Use latest nightly build of ML.NET Nuget package
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-libraries/nuget/v3/index.json"
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: Plotly.NET.CSharp, 0.0.1"
#r "nuget: Microsoft.ML.AutoML, 0.21.0-preview.23504.1"
#r "nuget: Microsoft.Data.Analysis, 0.21.0-preview.23504.1"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-libraries/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.21.0-preview.23504.1 Microsoft.ML.AutoML, 0.21.0-preview.23504.1 Plotly.NET.CSharp, 0.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `C:\Users\xiaoyuz\.nuget\packages\microsoft.ml.automl\0.21.0-preview.23504.1\interactive-extensions\dotnet\Microsoft.ML.AutoML.Interactive.dll`

Loading extensions from `C:\Users\xiaoyuz\.nuget\packages\microsoft.data.analysis\0.21.0-preview.23504.1\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `C:\Users\xiaoyuz\.nuget\packages\skiasharp\2.88.6\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

Loading extensions from `C:\Users\xiaoyuz\.nuget\packages\plotly.net.interactive\3.0.2\interactive-extensions\dotnet\Plotly.NET.Interactive.dll`

In [2]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.Data.Analysis;
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.AutoML.CodeGen;
using Microsoft.ML.Trainers.LightGbm;
using Microsoft.ML.Data;
using Plotly.NET;
using Microsoft.ML.Transforms.TimeSeries;
using Microsoft.ML.SearchSpace;
using System.Diagnostics;

#### Use `AutoTrainer` for built-in sweepable estimator candidates.

`AutoTrainer` provides built-in sweepable estimator candidates for binary-classification, multi-class classification and regression. For those scenarios, you can simply use those candidates instead of creating `SweepableEstimator` from scratch.

In [3]:
var context = new MLContext();
var regressionTrainerCandidates = context.Auto().Regression();
var binaryClassificationTrainerCandidates = context.Auto().BinaryClassification();
var multiclassClassificationTrainerCandidates = context.Auto().MultiClassification();

#### Use `AutoML().CreateSweepableEstimator` to create `SweepableEstimator`

In case the built-in `SweepableEstimator` doesn't satisfy your requirement, you can call `CreateSweepableEstimator` to create a customized `SweepableEstimator`. A `SweepableEstimator` is nothing different than a normal `Estimator` plus `SearchSpace`. The following code shows how to create a sweepable `LightGbm` and `SDCA`.

For simplicity, the built-in search space for `LightGbm` and `SDCA` is used but you can fully customize the search space however way you want. For more details on how to do that, please check [Parameter And SearchSpace](./Parameter%20and%20SearchSpace.ipynb)

In [4]:
var lgbmSearchSpace = new SearchSpace<LgbmOption>();
var sweepableLgbm = context.Auto().CreateSweepableEstimator((context, param) => {
    var option = new LightGbmRegressionTrainer.Options()
    {
        NumberOfLeaves = param.NumberOfLeaves,
        NumberOfIterations = param.NumberOfTrees,
        MinimumExampleCountPerLeaf = param.MinimumExampleCountPerLeaf,
        LearningRate = param.LearningRate,
        LabelColumnName = "Label",
        FeatureColumnName = "Features",
        Booster = new GradientBooster.Options()
        {
            SubsampleFraction = param.SubsampleFraction,
            FeatureFraction = param.FeatureFraction,
            L1Regularization = param.L1Regularization,
            L2Regularization = param.L2Regularization,
        },
        MaximumBinCountPerFeature = param.MaximumBinCountPerFeature,
    };

    return context.Regression.Trainers.LightGbm(option);
}, lgbmSearchSpace);

var sdcaSearchSpace = new SearchSpace<SdcaOption>();
var sweepableSdca = context.Auto().CreateSweepableEstimator((context, param) => {
    return context.Regression.Trainers.Sdca("Label", "Features", l1Regularization: param.L1Regularization, l2Regularization: param.L2Regularization);
}, sdcaSearchSpace);

#### Create `SweepablePipeline` with multiple trainer candidates.

`SweepablePipeline` allows you to put multiple estimators as candidates to a certain stage. During AutoML sweeping, these candidates will be evaluated seperatly and the one with best metric will be picked. Note that the estimator doesn't necessarily need to be a trainer, it can be a trainer, transformer or even a `SweepablePipeline`, as long as they all have the same input and output schema.

The following code shows how to create a `SweepablePipeline` with `sweepableSdca` and `sweepableLgbm` we created above.

In [5]:
var sweepablePipeline = context.Transforms.Concatenate("Features", "X1", "X2")
                            .Append(sweepableSdca, sweepableLgbm);

#### Config `AutoMLExperiment` using `sweepablePipeline`
In the next step, we are going to train `sweepablePipeline` on a generated non-linear dataset using `AutoMLExperiment`, which will sweeping both `sdca` and `lightGbm` on configured search space. Considering that `sdca` is a linear classifier, the winning model should be `lightGbm`.

In [6]:
var rand = new Random(0);
var context =new MLContext(seed: 1);
var x1 = Enumerable.Range(0, 1000).Select(_x => rand.NextSingle() * 100).ToArray();
var x2 = x1.Select(_x => rand.NextSingle() * 100).ToArray();
var y = Enumerable.Zip(x1, x2).Select(_x => _x.Second * _x.First + (rand.NextSingle() - 0.5f) * 10).ToArray();
var df = new DataFrame();
df["X1"] = DataFrameColumn.Create("X1", x1);
df["X2"] = DataFrameColumn.Create("X2", x2);
df["Label"] = DataFrameColumn.Create("Label", y);
var trainTestSplit = context.Data.TrainTestSplit(df);
df.Head(10)

index,X1,X2,Label
0,72.62433,1.8386767,137.49843
1,81.73254,63.497334,5192.4106
2,76.80227,74.96279,5752.4556
3,55.81612,24.123621,1349.5188
4,20.603315,56.236355,1159.8474
5,55.88848,35.49073,1982.7404
6,90.60271,65.19207,5902.1562
7,44.217785,89.708305,3963.569
8,97.754974,66.20816,6476.4365
9,27.370447,78.50348,2150.2874


In [7]:
var monitor = new NotebookMonitor(sweepablePipeline);
var experiment = context.Auto().CreateExperiment();
experiment.SetDataset(df, 5)
          .SetPipeline(sweepablePipeline)
          .SetTrainingTimeInSeconds(50)
          .SetRegressionMetric(RegressionMetric.RootMeanSquaredError)
          .SetMonitor(monitor);

// Configure Visualizer			
monitor.SetUpdate(monitor.Display());

var res = await experiment.RunAsync();

// check the type of last trainer for winning model, which should be lightGbm
(res.Model as TransformerChain<ITransformer>).Last().GetType()

index,Trial,Metric,Trainer,Parameters
0,0,1870.0494,Unknown,"{""_pipeline_"":{""_SCHEMA_"":""e0 * e1"",""e0"":{},""e1"":{""L1Regularization"":1,""L2Regularization"":0.1}},""_SCHEMA_"":""e0 * e1"",""e0"":{},""e1"":{""L1Regularization"":1,""L2Regularization"":0.1},""e2"":{""FeatureFraction"":1,""L1Regularization"":2E-10,""L2Regularization"":1,""LearningRate"":1,""MaximumBinCountPerFeature"":255,""MinimumExampleCountPerLeaf"":20,""NumberOfLeaves"":4,""NumberOfTrees"":4,""SubsampleFraction"":1}}"
1,1,705.6414,Unknown,"{""_pipeline_"":{""_SCHEMA_"":""e0 * e2"",""e0"":{},""e2"":{""FeatureFraction"":1,""L1Regularization"":2E-10,""L2Regularization"":1,""LearningRate"":1,""MaximumBinCountPerFeature"":254,""MinimumExampleCountPerLeaf"":20,""NumberOfLeaves"":4,""NumberOfTrees"":4,""SubsampleFraction"":1}},""_SCHEMA_"":""e0 * e1"",""e0"":{},""e1"":{""L1Regularization"":1,""L2Regularization"":0.1},""e2"":{""FeatureFraction"":1,""L1Regularization"":2E-10,""L2Regularization"":1,""LearningRate"":1,""MaximumBinCountPerFeature"":255,""MinimumExampleCountPerLeaf"":20,""NumberOfLeaves"":4,""NumberOfTrees"":4,""SubsampleFraction"":1}}"
2,2,2091.8748,Unknown,"{""_pipeline_"":{""_SCHEMA_"":""e0 * e1"",""e0"":{},""e1"":{""L1Regularization"":3.3867867,""L2Regularization"":0.051759623}},""_SCHEMA_"":""e0 * e1"",""e0"":{},""e1"":{""L1Regularization"":1,""L2Regularization"":0.1},""e2"":{""FeatureFraction"":1,""L1Regularization"":2E-10,""L2Regularization"":1,""LearningRate"":1,""MaximumBinCountPerFeature"":255,""MinimumExampleCountPerLeaf"":20,""NumberOfLeaves"":4,""NumberOfTrees"":4,""SubsampleFraction"":1}}"
3,3,107.595314,Unknown,"{""_pipeline_"":{""_SCHEMA_"":""e0 * e2"",""e0"":{},""e2"":{""FeatureFraction"":0.9930195610380121,""L1Regularization"":2E-10,""L2Regularization"":0.9999997766729865,""LearningRate"":0.4200893720564174,""MaximumBinCountPerFeature"":302,""MinimumExampleCountPerLeaf"":21,""NumberOfLeaves"":4,""NumberOfTrees"":838,""SubsampleFraction"":0.9999997766729865}},""_SCHEMA_"":""e0 * e1"",""e0"":{},""e1"":{""L1Regularization"":1,""L2Regularization"":0.1},""e2"":{""FeatureFraction"":1,""L1Regularization"":2E-10,""L2Regularization"":1,""LearningRate"":1,""MaximumBinCountPerFeature"":255,""MinimumExampleCountPerLeaf"":20,""NumberOfLeaves"":4,""NumberOfTrees"":4,""SubsampleFraction"":1}}"
4,4,90.599525,Unknown,"{""_pipeline_"":{""_SCHEMA_"":""e0 * e2"",""e0"":{},""e2"":{""FeatureFraction"":0.99999999,""L1Regularization"":3.676139172975011E-10,""L2Regularization"":0.9999997766729865,""LearningRate"":0.4230894791419352,""MaximumBinCountPerFeature"":390,""MinimumExampleCountPerLeaf"":21,""NumberOfLeaves"":4,""NumberOfTrees"":4511,""SubsampleFraction"":0.7160876722017678}},""_SCHEMA_"":""e0 * e1"",""e0"":{},""e1"":{""L1Regularization"":1,""L2Regularization"":0.1},""e2"":{""FeatureFraction"":1,""L1Regularization"":2E-10,""L2Regularization"":1,""LearningRate"":1,""MaximumBinCountPerFeature"":255,""MinimumExampleCountPerLeaf"":20,""NumberOfLeaves"":4,""NumberOfTrees"":4,""SubsampleFraction"":1}}"


Microsoft.ML.Data.RegressionPredictionTransformer<Microsoft.ML.Trainers.LightGbm.LightGbmRegressionModelParameters>

## Continue learning

> [⏩ Next Module - AutoML Tuner](./06-AutoML%20HPO%20and%20tuner.ipynb)  
> [⏪ Last Module - AutoML Sweepable API](./05%20-%20AutoML%20Sweepable%20API.ipynb)

## See also
- [Training and AutoML](./03-Training%20and%20AutoML.ipynb)
- [Regression with Taxi Dataset](./E2E-Regression%20with%20Taxi%20Dataset.ipynb)
- [Classification with Iris Dataset](./E2E-Classification%20with%20Iris%20Dataset.ipynb)
- [Kaggle with Titanic Dataset](./REF-Kaggle%20with%20Titanic%20Dataset.ipynb)